In [52]:
import csv
import pandas as pd
import numpy as np
import sklearn

from pymongo import MongoClient
global ALL_COLORS
ALL_COLORS = ['W', 'U', 'B', 'R', 'G']

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

In [74]:
client = MongoClient()
cards_en = client.scryfall.cards_en

In [59]:
def get_card(cards_en, card_name, set=None):
    card = cards_en.find_one({'name': card_name})

    if card is None:
        # separate this query since it will make it slower to combine
        query = {            
            'card_faces': {
                '$elemMatch': {
                    'name': card_name
                }
            },
        }

        card = cards_en.find_one(query)

    if card is None:
        error_message = 'No match for card name {}'.format(card_name)
        print(error_message)

    return(card)

In [54]:
names = []
with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    columns = next(draft_data)
    names = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]

In [71]:
def pip_vector(cards_en, card_name):
    card = get_card(cards_en, card_name)
    if card is None:
        print(card_name + ' Not Found')
    mana_cost = card['mana_cost']
    return np.array([mana_cost.count(f'{{{color}}}') for color in ALL_COLORS])  \
        + 1/2 * np.array([mana_cost.count(f'/{color}') for color in ALL_COLORS])\
        + 1/2 * np.array([mana_cost.count(f'{color}/') for color in ALL_COLORS])

pip_lookup = np.array([pip_vector(cards_en, name) for name in names])